In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import time
import os

from bs4 import BeautifulSoup
import re
import pandas as pd

import requests
import os
import urllib
from urllib.parse import urlparse

import subprocess


In [ ]:
# prompt_message = "Press Enter to naviagte to: "
prompt_message = None
DELAY_SECONDS_CONTENT_LOADING = 2

In [ ]:
IMPLICIT_DELAY = 60
EXPLICIT_DELAY = 30

In [ ]:
#setup option for chrome profile
chrome_options = Options()

## NEVER USE THE FOLLOWING OPTION SINCE IT WILL MAKE SUPPOSEDLY DIFFERENT WINDOWS FOR DIFFERENT CLASS MIXED IN SOME WAY!!!!
# chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
## the following option is to solve the error if using the above cause a problem
# chrome_options.add_argument("--remote-debugging-port=9222") 

prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
chrome_options.add_experimental_option("prefs", prefs)



#start web driver
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(IMPLICIT_DELAY)
wait = WebDriverWait(driver, EXPLICIT_DELAY)

In [ ]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, wait, xpath):

    try:
        wait.until(ec.presence_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element("xpath",xpath)

def sync_get_elements_by_xpath(driver, wait, xpath):

    try:
        wait.until(ec.presence_of_all_elements_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_elements("xpath",xpath)

In [ ]:
def check_lesson_complete(item)->bool:
    e1 = item.find_element_by_xpath("./div/div")
    c = e1.get_attribute('class')
    if c == 'index--lesson-card--mwX1V index--card--3DZMr shared--card--3X88h':
        return True
    else:
        return False

def expand_all_complete_lessons():
    #click open the ones that were marked Complete
    lessons = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")
    for lesson in lessons:
        if check_lesson_complete(lesson):
            lesson.click()
    element = driver.find_element_by_xpath("//*[h3='lesson 1']")
    driver.execute_script("return arguments[0].scrollIntoView(true);", element)
#     time.sleep(1)

def embed_tag(tag, html_str):
    return '<{}>'.format(tag) + html_str + '</{}>'.format(tag)

def debug_breakpoint(title, message=None):
    if message == None:
        print(title)
    else:
        input(title + ' - ' + message)

def write_to_clipboard(output):
    process = subprocess.Popen(
        'pbcopy', env={'LANG': 'en_US.UTF-8'}, stdin=subprocess.PIPE)
    process.communicate(output.encode('utf-8'))

In [ ]:
def download_css_files():
    
    css_folder_path = os.path.join(root_folder, topic_name,lesson_name, part_name, "css")
    
    if not os.path.exists(css_folder_path):
        os.makedirs(css_folder_path)
        
    html_src = driver.page_source
    base = driver.current_url
    soup = BeautifulSoup(html_src)
    css_link_elms = soup.find_all('link', {'rel': 'stylesheet'})
    
    css_links = []
    for idx, css_link_elm in enumerate(css_link_elms, start = 1):
        link = css_link_elm['href']
        if link.startswith('/'):
            link = urllib.parse.urljoin(base, link)
        css_links.append(link)
#         print(idx, ": ", link)
    
    for css_link in css_links:
        r = requests.get(css_link, allow_redirects=True)
        a = urlparse(css_link)
        file_name = os.path.basename(a.path)
        file_name = os.path.join(css_folder_path, file_name)
#         print("file_name= ", file_name) 
    
        open(file_name, 'wb').write(r.content)

def make_images_local(soup):
    
    images_folder = os.path.join(root_folder, topic_name,lesson_name, part_name, "img")

    img_elms = soup.find_all('img')
    for img_elm in img_elms:
        img_link = img_elm['src']
        r = requests.get(img_link, allow_redirects=True)
        a = urlparse(img_link)
        file_name = os.path.basename(a.path)                
        img_path = "./" + images_folder + "/" + file_name

        if not os.path.exists(images_folder):
            os.makedirs(images_folder)

        open(img_path, 'wb').write(r.content)
#         print("file_name: ", file_name)
        # replace src in img element
        img_elm['src'] = "./img/{}".format(file_name)

def create_head_str():
    
    css_folder_path = os.path.join(root_folder, topic_name,lesson_name, part_name, "css")
    css_filenames = [f for f in os.listdir(css_folder_path) if os.path.isfile(os.path.join(css_folder_path, f))]
    
    page_html = driver.page_source
    page_soup = BeautifulSoup(page_html, 'html.parser')
    style_elms = page_soup.find('head').find_all('style')
    
    meta_elm = '<meta http-equiv="Content-type" content="text/html; charset=utf-8">'
    head_str = '<head>' + meta_elm
    for css_filename in css_filenames:
        link_str = '<link rel="stylesheet" href="./css/{}">'.format(css_filename)
        head_str = head_str + link_str
    for style_elm in style_elms:
        head_str = head_str + str(style_elm)
    
    head_str = head_str + '</head>'
#     print("head_str: ", head_str)
    return head_str

def process_content_html(content_html_str):
    
    download_css_files()
            
    head_str = create_head_str()
    body_str = embed_tag('body', content_html_str)
    page_html_str = head_str + body_str
    html_str = embed_tag('html', page_html_str)
    
    # remove unwanted div element
    soup = BeautifulSoup(html_str, 'html.parser')
    div_to_be_remove_class_name = '_main--notification-bar--2AVbT'
    to_be_removed_elms = soup.find_all('div',class_=div_to_be_remove_class_name)
    for to_be_removed_elm in to_be_removed_elms:
        to_be_removed_elm.decompose()
    
    make_images_local(soup)
    
    return soup.prettify()

def copy_youtube_link_to_clipboard(video_link):
    write_to_clipboard(video_link)
    time.sleep(1)
    
def grab_all_youtube_links(content_html_str, download=False):

    content_soup = BeautifulSoup(content_html_str, 'html.parser')
    youtube_iframe_elms = content_soup.find_all('iframe', {'title': 'YouTube video player'})
    video_links = ''
    for idx, youtube_iframe_elm in enumerate(youtube_iframe_elms, start=1):
        video_link = youtube_iframe_elm['src']
        video_link = video_link + '#index={}_{}_{}_{}'.format(topic_name,lesson_name, part_name,idx)
        video_links = video_links + video_link + '\n\n'
        if download == True:
            copy_youtube_link_to_clipboard
        
    #     print(idx, ' : ', video_link)
#     print(video_links)
    with open('all_youtube_video_links.txt','a') as fp:
        fp.writelines(video_links)

In [ ]:
def save_processed_html():

    contents = sync_get_elements_by_xpath("//div[@id='content']")
    assert len(contents) == 1

    content_html = contents[0].get_attribute('outerHTML')
    html_str = process_content_html(content_html)
    grab_all_youtube_links(content_html, False)
    
    file_name = os.path.join(root_folder, topic_name,lesson_name, part_name, "content.html")
    with open(file_name, 'w') as f:
        f.write(html_str)
#     print("html_str= ", html_str)

In [ ]:
root_address = 'https://learn.udacity.com/nanodegrees/nd880/parts/cd0569'
root_folder = './ai_for_trading_course_downloaded'

In [ ]:
# start
driver.get(root_address)
os.makedirs(root_folder, exist_ok=True)

In [ ]:
sync_get_element_by_xpath(driver, wait, "//div[text()='Sign in']").click()

email = sync_get_element_by_xpath(driver, wait, "//input[@id='email']")
password = sync_get_element_by_xpath(driver, wait, "//input[@id='revealable-password']")

email.send_keys("huohsien@gmail.com")
password.send_keys("Huo18941256")

sync_get_element_by_xpath(driver, wait, "//div[@data-testid='signin-form']//span[text()='Sign in']/..").click()


In [ ]:
# def sign_in():
#     email = sync_get_element_by_xpath("//input[@data-testid='signin-email']")
#     password = sync_get_element_by_xpath("//input[@data-testid='signin-password']")

#     email.send_keys("huohsien@gmail.com")
#     password.send_keys("jj1216")

# #     wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb")))
# #     sign_in_btn = driver.find_element_by_class_name("button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb") 
#     sign_in_btn = sync_get_element_by_xpath("//span[contains(text(), 'Sign In')]/..")
#     sign_in_btn.click()

# Create a notbook for scraping AI for Trading course from an old notebook called "dlnd_course_scrape-V3.ipynb" Rest for 2024/3/24

In [ ]:
# Prepare loop through all topics
topic_elms = sync_get_elements_by_xpath("//*[@id='tree-core-curriculum']/*")
num_topics = len(topic_elms)

# iterate each topic
for idx_topics in range(num_topics):
    
    # Create topic folder
    topic_name = ''.join(topic_elms[idx_topics].text.split('\n')[0:2])                   
    os.makedirs(os.path.join(root_folder, topic_name), exist_ok=True)
    
    debug_breakpoint("Topic {}".format(topic_name), prompt_message)
    
    # click into a topic
    link_topic = topic_elms[idx_topics].find_element_by_xpath("./a")
    link_topic.click()
    
    # Prepare loop through all lessons
    expand_all_complete_lessons()
    
    lesson_elms = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")
    num_lessons = len(lesson_elms)
    
    #iterate each lesson
    for idx_lesson in range(num_lessons):

        # Create lesson folder
        lesson_elm = lesson_elms[idx_lesson]
        lesson_name = ' - '.join(lesson_elm.text.split('\n')[1:3])                   
        os.makedirs(os.path.join(root_folder, topic_name,lesson_name), exist_ok=True)

        debug_breakpoint("Lesson {}".format(lesson_name), prompt_message)

        # click into a lesson
        link_lesson = lesson_elm.find_element_by_xpath("./div/div/a")
        link_lesson.click()
        
        # Prepare loop through all parts of a lesson
        part_elms = sync_get_elements_by_xpath("//*[@id='tree-concepts']/ol/*")
        num_parts = len(part_elms)
        
        # iterate each part
        for idx_part in range(num_parts):
            
            # Create part folder
            part_elm = part_elms[idx_part]
            part_name = part_elm.text
            os.makedirs(os.path.join(root_folder, topic_name,lesson_name, part_name), exist_ok=True)
            
            debug_breakpoint("Lesson part {}".format(part_name), prompt_message)
            
            # click into a part
            link_part = part_elm.find_element_by_xpath("./a")
            link_part.click()
            
            # REAL CONTENTS HERE to Be STORED!
            #
            time.sleep(DELAY_SECONDS_CONTENT_LOADING)
            
            save_processed_html()
#             debug_breakpoint(driver.current_url, prompt_message)
            
            # reflesh webElement after webpage reload
            part_elms = sync_get_elements_by_xpath("//*[@id='tree-concepts']/ol/*")

        # go back to Lesson List
        back_link = sync_get_element_by_xpath("//h3[@class='_sidebar--header--2BaH_']/a") 
        back_link.click()
        
        # click open the completed lessons
        expand_all_complete_lessons()
        
        # reflesh webElement after webpage reload
        lesson_elms = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")

    # reflesh webElement after webpage reload
    topic_elms = sync_get_elements_by_xpath("//*[@id='tree-core-curriculum']/*")
    

In [ ]:
# link_topic.click() 

In [ ]:
# expand_all_complete_lessons()



In [ ]:
# element = driver.find_element_by_xpath("//*[h3='lesson 1']")
# driver.execute_script("return arguments[0].scrollIntoView(true);", element)